In [ ]:
from google.colab import files
uploaded = files.upload()

Saving shakespeare.txt to shakespeare.txt


# 1. 데이터 다듬기

In [ ]:
import os, re 
import numpy as np
import tensorflow as tf

# 파일을 읽기모드로 열고
# 라인 단위로 끊어서 list 형태로 읽어옵니다.
file_path = ('shakespeare.txt')
with open(file_path, "r") as f:
    raw_corpus = f.read().splitlines()

# 앞에서부터 10라인만 화면에 출력해 볼까요?
for i, txt in enumerate(raw_corpus[:9]):
    print(f'sentence[{i}]', f'"{txt}"')
# print(raw_corpus[:9])

sentence[0] "First Citizen:"
sentence[1] "Before we proceed any further, hear me speak."
sentence[2] ""
sentence[3] "All:"
sentence[4] "Speak, speak."
sentence[5] ""
sentence[6] "First Citizen:"
sentence[7] "You are all resolved rather to die than to famish?"
sentence[8] ""


- 실습에 사용할 라이브러리를 불러온다.
- 준비한 txt 파일의 내용을 불러온다.

In [ ]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: # 길이가 0인 문장은 건너뜀
        continue
    if sentence[-1] == ":": # 문장의 끝이 : 인 문장은 건너뜀
        continue
    if idx > 9: # 문장 10개만 확인해보자
        break

    print(sentence)

Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?


- 원치 않는 문장인 **화자가 표기된 문장(0, 3, 6)**, 그리고 **공백인 문장(2, 5, 9)**
- 화자가 표기된 문장의 끝은 `:`으로 끝나고 일반적으로 대사가 끝날때 `:`로 끝나지 않으니 `:`를 기준으로 문장 제외

In [ ]:
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    # 1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
    sentence = sentence.lower().strip()
    # 2. 특수문자 양쪽에 공백을 넣고
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    # 3. 여러개의 공백은 하나의 공백으로 바꿉니다
    sentence = re.sub(r'[" "]+', " ", sentence)
    # 4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)
    # 5. 다시 양쪽 공백을 지웁니다
    sentence = sentence.strip()
    #  6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
    sentence = '<start> ' + sentence + ' <end>'
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


##### **토큰화(Tokenize)**: 텍스트 생성 모델은 단어 사전을 만들어서 문장을 일정한 기준으로 쪼개주는 것
- 가장 쉬운 방법은 띄어쓰기 기준으로 나누기가 있지만 약간의 문제가 있다.
1. Hi, my name is John. *("Hi," "my", ..., "john." 으로 분리됨) - 문장부호
2. First, open the first chapter. *(First와 first를 다른 단어로 인식) - 대소문자
3. He is a ten-year-old boy. *(ten-year-old를 한 단어로 인식) - 특수문자
- 1번을 막기 위해 **문장 부호 양쪽에 공백 추가**
- 2번을 막기 위해 **모든 문자들을 소문자로 변환**
- 3번을 막기 위해 **특수문자들은 제거**
***
- 우리가 구축해야 할 데이터셋은 어떤 모양일까?
```
언어 모델의 입력 문장 : <start> 난는 밥을 먹었다
언어 모델의 출력 문장 : 나는 밥을 먹었다 <end>
```
> **소스 문장(Source Semtemce)**: 자연어처리 분야에서 모델에 입력되는 문장(X_train)    
**타겟 문장(Target Sentence)**: 정답 역할을 하게 될 모델의 출력 문장(y_train)



In [ ]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> before we proceed any further , hear me speak . <end>',
 '<start> speak , speak . <end>',
 '<start> you are all resolved rather to die than to famish ? <end>',
 '<start> resolved . resolved . <end>',
 '<start> first , you know caius marcius is chief enemy to the people . <end>',
 '<start> we know t , we know t . <end>',
 '<start> let us kill him , and we ll have corn at our own price . <end>',
 '<start> is t a verdict ? <end>',
 '<start> no more talking on t let it be done away , away ! <end>',
 '<start> one word , good citizens . <end>']

- `<end>`를 없애면 **소스문장**
- `<start>`를 없애면 **타겟문장**
> 인공지능의 모국어인 숫자로 표현하기위해 **배우고자 하는 언어**를 **모국어로 표현**

In [ ]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  143   40 ...    0    0    0]
 [   2  110    4 ...    0    0    0]
 [   2   11   50 ...    0    0    0]
 ...
 [   2  149 4553 ...    0    0    0]
 [   2   34   71 ...    0    0    0]
 [   2  945   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f7c0ac5ba50>


- tensorflow는 자연어 처리를 위한 여러 가지 모듈을 제공
- `tf.keras.preprocessing.text.Tokenizer`패키지 이용   
    - 이 패키지는 정제된 데이터를 토큰화 시켜줌  
    - 단어사전(vocabulary 또는 dictionary)을 만들어줌   
    - 데이터를 숫자로 변환시켜줌   
     

- 이 과정을 **벡터화(vectorize)**라 하며, 숫자로 변환된 데이터를 **텐서(tensor)**라 한다.
> 텐서(tensor)는 어려운 물리학 및 수학 개념이다. 아래의 링크를 통해 간단한 개념을 알고가자.   
[Tensor란 무엇인가?](https://rekt77.tistory.com/102)
    

In [ ]:
print(tensor[:3, :10])

[[   2  143   40  933  140  591    4  124   24  110]
 [   2  110    4  110    5    3    0    0    0    0]
 [   2   11   50   43 1201  316    9  201   74    9]]


- 생성된 텐서 데이터를 3번째 행, 10번째 열까지만 출력해보자.
- 텐서 데이터는 모두 정수로 이루어짐
- tokenizer에 구축된 단어 사전의 인덱스임

In [ ]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : .
6 : the
7 : and
8 : i
9 : to
10 : of


- tokenizer의 단어사전은 어떻게 구축되어 있는지 확인해보자.
- 2번 인덱스가 <start>이다.
- `tensor[:3, :10]`의 시작이 왜 2로 시작하는지 이해할 수 있겠다.


In [ ]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2 143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0
   0   0]
[143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0   0
   0   0]


- 생성된 텐서를 **소스**와 **타겟**으로 분리하여 모델이 학습할 수 있게 해주자.
- 텐스 출력부에서 행 뒤쪽에 0이 많이 나온 부분은 정해진 입력 시퀀스 길이보다 문장이 짧을 경우 0으로 패팅(padding)을 채워 넣은 것이다.
- 사전에는 없지만 0은 바로 패딩 문자 `<pad>`이다.
***
- 변수로 지정한 `corpus` 내의 첫 번째 문장의 소스와 타겟 문장을 확인해보자.   
    - **소스**: 2`<start>`에서 시작해서 3`<end>`으로 끝난 후 0`<pad>`로 채워짐   
    - **타겟**: 2로 시작하지 않고 소스를 왼쪽으로 한 칸 시프트 한 형태를 가짐

In [ ]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 20), (256, 20)), types: (tf.int32, tf.int32)>

**마지막으로 데이터셋 객체 생성**   
**keras의 경우 model.fit(x_train, y_train,....)형태로 학습을 진행하였다**   
**하지만 텐서플로우를 활용할 경우 텐서로 생성된 데이터를 이용해 tf.data.Dataset 객체를 생성하는 방법을 사용**
- `tf.data.Dataset`객체는 텐서플로우에서 사용할 경우 데이터 입력 파이프라인을 통한 속도 개선 및 각종 편의 기능을 제공
- `tf.data.Dataset.from_tensor_slices()` 메소드를 이용해 `tf.data.Dataset`객체를 생성


# 2. 인공지능 학습시키기
> 인공지능이라 부르는 것은 인공신경망이자 딥러닝 네트워크이자 순환신경망이자.......   
그냥 모델(model)이라 부르자.   
![RNN](C:\Users\zzocojoa\Desktop\pic_2.png)

In [ ]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

- 입력 텐서에는 단어 사전의 인덱스가 들어있다고 했다.
- Embedding레이어에는 이 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔준다.
- 이 워드 벡터는 의미 공간에서 단어의 추상적 표현(representation)으로 사용된다.
***
- 위 코드에서 `embeddin_size`는 워드 벡터의 차원수 즉, 다어가 추상적으로 표현되는 크기이다.   
(만약 크기가 2일 경우)   
    - 차갑다: [0.0, 1.0]   
    - 뜨겁다: [1.0, 0.0]   
    - 미지근하다: [0.5, 0.5]
- 커질수록 단어의 추상적인 특징들을 더 잡아낼 수 있지만, 그만큼 충분한 데이터가 주어지지 않으면 오히려 혼란을 만든다.
- 이번 모델에서는 `embeddin_size`는 256이 적당하다.
***
- LSTM 레이어의 `hidden state` 의 차원수인 hidden_size 도 같은 맥락이다.   
    - hidden_size 는 모델에 얼마나 많은 일꾼을 둘 것인가.   
    - 일꾼들은 모두 같은 데이터를 보고 각자의 생각을 가지는데,   
    충분한 데이터가 주어지면 올바른 결정을 내리겠지만 그렇지 않으면 배가 산으로 갈 뿐
- 이번 모델에서 `hidden state`는 1024가 적당하다.

In [ ]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 20, 7001), dtype=float32, numpy=
array([[[-8.7577799e-05,  7.7194483e-05, -5.9036476e-05, ...,
          2.3434141e-04,  5.9491878e-05, -2.5918504e-04],
        [-4.1279459e-04,  1.9561421e-04, -4.2430585e-04, ...,
          1.8878248e-04, -4.1644086e-04, -1.3420604e-04],
        [-5.0915993e-04,  2.3712967e-04, -7.9280575e-04, ...,
          2.1559636e-04, -6.5501564e-04,  1.8159240e-04],
        ...,
        [ 1.9092321e-03, -2.2421991e-03,  3.3173070e-03, ...,
          9.0502016e-04, -2.6100739e-03,  5.9498032e-03],
        [ 1.9192731e-03, -2.5304379e-03,  3.4943561e-03, ...,
          8.9233613e-04, -2.8159029e-03,  6.4732819e-03],
        [ 1.9202124e-03, -2.7837725e-03,  3.6535128e-03, ...,
          8.9514215e-04, -2.9803335e-03,  6.9239838e-03]],

       [[-8.7577799e-05,  7.7194483e-05, -5.9036476e-05, ...,
          2.3434141e-04,  5.9491878e-05, -2.5918504e-04],
        [-3.1337997e-05,  4.6573853e-04,  2.8920374e-04, ...,
          2.4093908e-05,  

- model에 데이터를 조금 태워 보는 것
- input shape가 결정되면서 model.build()가 자동으로 호출
***
- 텐서 shape를 보면 `shape=(256, 20, 7001)`을 알 수 있다.
- 7001은 Dense레이어의 출력 차원수
    - 7001개의 단어 중 어느 단어의 확률이 가장 높을지를 모델링
- 256은 이전 스텝에서 지정한 배치 사이즈   
    - `dataset.take(1)`를 통해서 1개의 배치, 즉 256개의 문장 데이터를 가져온 것
- 20: LSTM은 자신에게 입력된 시퀀스의 길이만큼 동일한 길이의 시퀀스를 출력   
    - `tf.keras.layers.LSTM(hidden_size, return_sequences=True)`로 호출한 LSTM 레이어에서 `return_sequences=True`이라고 지정한 부분
***
- 모델의 입력 데이터의 시퀀스 길이가 얼마인지 모르고, 모델을 만들면서 알려준 적도 없다.   
그럼 어떻게 알게된 것일까??   
    > 데이터를 입력받으면서 알게된 것이다. 데이터셋의 max_len이 20으로맞춰져 있던 것이다.

In [ ]:
model.summary()

Model: "text_generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  1792256   
                                                                 
 lstm (LSTM)                 multiple                  5246976   
                                                                 
 lstm_1 (LSTM)               multiple                  8392704   
                                                                 
 dense (Dense)               multiple                  7176025   
                                                                 
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


- Output Shape를 정확하게 알려주지 않는다.
- 이 모델은 입력 시퀀스의 길이를 모르기 때문에 Output Shape를 특정할 수 없다.
- 하지만 모델의 파라미터 사이즈는 측정된다.(대략 22million)


- Loss: 모델이 오답을 만들고 있는 정도
- 오답률이 감소할 수 록 **학습이 잘 진행되고 있다.**

In [ ]:
# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
93/93 [==============================] - 44s 431ms/step - loss: 3.4686
Epoch 2/30
93/93 [==============================] - 40s 430ms/step - loss: 2.7954
Epoch 3/30
93/93 [==============================] - 40s 430ms/step - loss: 2.6776
Epoch 4/30
93/93 [==============================] - 40s 431ms/step - loss: 2.5781
Epoch 5/30
93/93 [==============================] - 40s 430ms/step - loss: 2.5176
Epoch 6/30
93/93 [==============================] - 40s 428ms/step - loss: 2.4620
Epoch 7/30
93/93 [==============================] - 40s 426ms/step - loss: 2.3990
Epoch 8/30
93/93 [==============================] - 40s 428ms/step - loss: 2.3417
Epoch 9/30
93/93 [==============================] - 40s 429ms/step - loss: 2.2851
Epoch 10/30
93/93 [==============================] - 40s 427ms/step - loss: 2.2346
Epoch 11/30
93/93 [==============================] - 40s 431ms/step - loss: 2.1814
Epoch 12/30
93/93 [==============================] - 40s 428ms/step - loss: 2.1308
Epoch 13/30
9